<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_01/14%20-%20Atividade%20Avaliativa/C2W1_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Uso de CNNs com o conjunto de dados Cats vs Dogs

Nesta atividade você usará o famoso conjunto de dados `Cats vs Dogs` para treinar um modelo que possa classificar imagens de cães de imagens de gatos. Para isso, você criará sua própria rede neural convolucional no Tensorflow e aproveitará os utilitários de pré-processamento de imagens do Keras.

Você também criará algumas funções auxiliares para mover as imagens pelo sistema de arquivos, portanto, se não estiver familiarizado com o módulo `os`, não deixe de dar uma olhada em [_os — Miscellaneous operating system interfaces_](https://docs.python.org/3/library/os.html).

Vamos começar!

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

Faça o download do conjunto de dados de sua fonte original executando a célula abaixo. 

Observe que o arquivo `zip` que contém as imagens é descompactado no diretório `/tmp`.

In [ ]:
# Se o URL não funcionar, acesse https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# E clique com o botão direito do mouse no link "Download Manually" para obter um novo URL para o conjunto de dados

# Observação: esse conjunto de dados é muito grande e o download levará algum tempo

!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Agora as imagens estão armazenadas no diretório `/tmp/PetImages`. Há um subdiretório para cada classe, portanto, um para cães e outro para gatos.

In [ ]:

source_path = '/tmp/PetImages'

source_path_dogs = os.path.join(source_path, 'Dog')
source_path_cats = os.path.join(source_path, 'Cat')

# Exclui todos os arquivos que não sejam de imagem (há dois arquivos .db incluídos no conjunto de dados)
!find /tmp/PetImages/ -type f ! -name "*.jpg" -exec rm {} +

# os.listdir retorna uma lista contendo todos os arquivos sob o caminho fornecido
print(f"Há {len(os.listdir(source_path_dogs))} imagens de cães.")
print(f"Há {len(os.listdir(source_path_cats))} imagens de gatos.")

**Resultado esperado:**

```
Há 12500 imagens de cães.
Há 12500 imagens de gatos.
```

Você precisará de um diretório para cats-v-dogs e subdiretórios para treinamento
e validação. Esses, por sua vez, precisarão de subdiretórios para "cats" e "dogs". Para isso, complete o comando `create_train_val_dirs` abaixo:

In [ ]:
# Definir o diretório raiz
root_dir = '/tmp/cats-v-dogs'

# Diretório vazio para evitar FileExistsError se a função for executada várias vezes
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

def create_train_val_dirs(root_path):
  """
  Cria diretórios para os conjuntos de treinamento e teste
  
  Args:
    root_path (string) - o caminho do diretório base para criar subdiretórios
  
  Retorna:
    Nenhum
  """
  ### COMECE O CÓDIGO AQUI

  # DICA:
  # Use os.makedirs para criar seus diretórios com subdiretórios intermediários
  # Não codifique os caminhos. Use os.path.join para anexar os novos diretórios ao parâmetro root_path


  pass
  
  ### TERMINE O CÓDIGO AQUI

  
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("Você não deveria estar vendo isso, pois o diretório superior é removido antes")

In [ ]:
# Teste sua função create_train_val_dirs

for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

**Saída esperada (a ordem dos diretórios pode variar):**

``` txt
/tmp/cats-v-dogs/training
/tmp/cats-v-dogs/validation
/tmp/cats-v-dogs/training/cats
/tmp/cats-v-dogs/training/dogs
/tmp/cats-v-dogs/validation/cats
/tmp/cats-v-dogs/validation/dogs

```

Codifique a função `split_data` que recebe os seguintes argumentos:
- SOURCE_DIR: diretório que contém os arquivos

- TRAINING_DIR: diretório para o qual uma parte dos arquivos será copiada (será usado para treinamento)
- VALIDATION_DIR: diretório para o qual uma parte dos arquivos será copiada (será usado para validação)
- SPLIT_SIZE: determina a parte das imagens usadas para treinamento.

Os arquivos devem ser randomizados, de modo que o conjunto de treinamento seja uma amostra aleatória dos arquivos e o conjunto de validação seja composto pelos arquivos restantes.

Por exemplo, se `SOURCE_DIR` for `PetImages/Cat` e `SPLIT_SIZE` for .9, 90% das imagens em `PetImages/Cat` serão copiadas para o diretório `TRAINING_DIR
e 10% das imagens serão copiadas para o diretório `VALIDATION_DIR`.

Todas as imagens devem ser verificadas antes da cópia, portanto, se o tamanho do arquivo for zero, elas serão omitidas do processo de cópia. Se esse for o caso, sua função deverá imprimir uma mensagem como `"<NOME DO ARQUIVO> tem tamanho zero, portanto será ignorada."`. **Você deve executar essa verificação antes da divisão para que apenas as imagens diferentes de zero sejam consideradas ao fazer a divisão real.**


Dicas:

- `os.listdir(DIRECTORY)` retorna uma lista com o conteúdo desse diretório.

- `os.path.getsize(PATH)` retorna o tamanho do arquivo

- `copyfile(source, destination)` copia um arquivo da origem para o destino

- `random.sample(list, len(list))` embaralha uma lista

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Divide os dados em conjuntos de treinamento e teste
  
  Args:
    SOURCE_DIR (string): caminho do diretório que contém as imagens
    TRAINING_DIR (string): caminho do diretório a ser usado para treinamento
    VALIDATION_DIR (string): caminho do diretório a ser usado para validação
    SPLIT_SIZE (float): proporção do conjunto de dados a ser usado para treinamento
    
  Retorna:
    Nenhum
  """

  ### COMECE O CÓDIGO AQUI

  pass

  ### TERMINE O CÓDIGO AQUI

In [ ]:
# Teste sua função split_data

# Definir caminhos
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"

TRAINING_DIR = "/tmp/cats-v-dogs/training/"
VALIDATION_DIR = "/tmp/cats-v-dogs/validation/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

# Diretórios vazios para o caso de você executar essa célula várias vezes
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_CATS_DIR)) > 0:
  for file in os.scandir(VALIDATION_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DOGS_DIR)) > 0:
  for file in os.scandir(VALIDATION_DOGS_DIR):
    os.remove(file.path)

# Definir a proporção de imagens usadas para treinamento
split_size = .9

# Executar a função
# OBSERVAÇÃO: as mensagens sobre imagens de comprimento zero devem ser impressas
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

# Verifique se o número de imagens corresponde à saída esperada

# Sua função deve realizar cópias em vez de mover imagens, portanto, os diretórios originais devem conter imagens inalteradas
print(f"\n\nO diretório original de gatos tem {len(os.listdir(CAT_SOURCE_DIR))} imagens")
print(f"O diretório original do cães tem {len(os.listdir(DOG_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"Há {len(os.listdir(TRAINING_CATS_DIR))} imagens de gatos para treinamento")
print(f"Há {len(os.listdir(TRAINING_DOGS_DIR))} imagens de cães para treinamento")
print(f"Há {len(os.listdir(VALIDATION_CATS_DIR))} imagens de gatos para validação")
print(f"Há {len(os.listdir(VALIDATION_DOGS_DIR))} imagens de cães para validação")

**Saída Esperada:**

```
666.jpg tem tamanho zero, portanto será ignorada.
11702.jpg tem tamanho zero, portanto será ignorada.


O diretório original de gatos tem  12500 imagens
O diretório original de cães tem 12500 imagens

Há 11249 imagens de gatos para treinamento
Há 11249 imagens de cães para treinamento
Há 1250 imagens de gatos para validação
Há 1250 imagens de cães para validação
```

Agora que você organizou os dados de uma forma que pode ser facilmente alimentada pelo `ImageDataGenerator` do Keras, é hora de codificar os geradores que produzirão lotes de imagens, tanto para treinamento quanto para validação. Para isso, complete a função `train_val_generators` abaixo.

É importante observar que as imagens desse conjunto de dados vêm em uma variedade de resoluções. Felizmente, o método `flow_from_directory` permite que você padronize isso definindo uma tupla chamada `target_size` que será usada para converter cada imagem para essa resolução de destino. **Para este exercício, use um `target_size` de (150, 150)**.

**Dica** 

Não use o aumento de dados definindo parâmetros extras ao instanciar a classe `ImageDataGenerator`. Isso fará com que o treinamento do seu modelo demore mais para atingir o limite de acurácia necessário para ser aprovado neste trabalho, e esse tópico será abordado na próxima atividade.

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Cria os geradores de dados de treinamento e validação
  
  Args:
    TRAINING_DIR (string): caminho do diretório que contém as imagens de treinamento
    VALIDATION_DIR (string): caminho do diretório que contém as imagens de teste/validação
    
  Retorna:
    train_generator, validation_generator - tupla contendo os geradores
  """
  ### COMECE O CÓDIGO AQUI

  # Instanciar a classe ImageDataGenerator (não se esqueça de definir o argumento rescale)
  train_datagen = None

  # Passe os argumentos apropriados para o método flow_from_directory
  train_generator = train_datagen.flow_from_directory(directory=None,
                                                      batch_size=None,
                                                      class_mode=None,
                                                      target_size=(None, None))

  # Instanciar a classe ImageDataGenerator (não se esqueça de definir o argumento rescale)
  validation_datagen = None

  # Passe os argumentos apropriados para o método flow_from_directory
  validation_generator = validation_datagen.flow_from_directory(directory=None,
                                                                batch_size=None,
                                                                class_mode=None,
                                                                target_size=(None, None))
  ### TERMINE O CÓDIGO AQUI
  return train_generator, validation_generator

In [ ]:
# Teste seus geradores
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

**Saída Esperada:**

```
Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
```


Uma última etapa antes do treinamento é definir a arquitetura do modelo que será treinado.

Complete a função `create_model` abaixo, que deve retornar um modelo `Sequential` do Keras.

Além de definir a arquitetura do modelo, você também deve compilá-lo. Portanto, certifique-se de usar uma função `loss` que seja compatível com o `class_mode` definido no exercício anterior, que também deve ser compatível com a saída da sua rede. Você poderá saber se elas não são compatíveis se receber um erro durante o treinamento.

**Observe que você deve usar pelo menos 3 camadas de convolução para obter o desempenho desejado.

In [ ]:
def create_model():
  # DEFINIR UM MODELO KERAS PARA CLASSIFICAR GATOS E CACHORROS
  # USE PELO MENOS 3 CAMADAS DE CONVOLUÇÃO

  ### COMECE O CÓDIGO AQUI

  model = tf.keras.models.Sequential([ 
      None,
  ])

  
  model.compile(optimizer=None,
                loss=None,
                metrics=['accuracy']) 
    
  ### TERMINE O CÓDIGO AQUI

  return model

Agora é hora de treinar seu modelo!

**Nota:** Você pode ignorar os avisos `UserWarning: Possivelmente dados EXIF corrompidos.`.

In [ ]:
# Obter o modelo não treinado
model = create_model()

# Treinar o modelo
# Observe que isso pode levar algum tempo.
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

Após o término do treinamento, você pode executar a seguinte célula para verificar a acurácia do treinamento e da validação obtida no final de cada época.

**Para ser aprovado nesta tarefa, seu modelo deve atingir uma acurácia de treinamento de pelo menos 95% e uma acurácia de validação de pelo menos 80%**. Se o seu modelo não atingiu esses limites, tente treinar novamente com uma arquitetura de modelo diferente e lembre-se de usar pelo menos 3 camadas convolucionais.

In [ ]:
#-----------------------------------------------------------
# Recupere uma lista de resultados de lista em dados de treinamento e teste
# conjuntos para cada época de treinamento
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Obter o número de épocas

#------------------------------------------------
# Plotar a acurácia do treinamento e da validação por época
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Acurária de Treino")
plt.plot(epochs, val_acc, 'b', "Acurácia de Validação")
plt.title('Acurácia do treinamento e da validação')
plt.show()
print("")

#------------------------------------------------
# Plotar a perda de treinamento e validação por época
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Perda de treinamento")
plt.plot(epochs, val_loss, 'b', "Perda de validação")
plt.show()

Você provavelmente perceberá que o modelo com _overfitting_, o que significa que ele está fazendo um ótimo trabalho na classificação das imagens no conjunto de treinamento, mas tem dificuldades com os novos dados. Isso é perfeitamente normal e você aprenderá a atenuar esse problema na próxima atividade.

**Parabéns por terminar essa tarefa!**

Você implementou com sucesso uma rede neural convolucional que classifica imagens de gatos e cachorros, juntamente com as funções auxiliares necessárias para pré-processar as imagens!

**Continue assim!